# Data

In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric, Dataset

In [2]:
df = pd.read_csv('data/annotation task/comMid2k_Shuffled_no_none.tsv',sep='\t')
df_dict = df.apply(lambda x:{"x":x['text'],"y":x['mitigated_text']}, axis=1)
df = pd.DataFrame(df_dict, columns=['mitigation'])
raw_dataset = Dataset.from_pandas(df)

In [3]:
metric = load_metric("sacrebleu")

/var/folders/28/xsb_n5f52wb0yymcyysnq0980000gn/T/ipykernel_94634/3247265623.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [4]:
split_datasets = raw_dataset.train_test_split(test_size=0.15) 

In [5]:
split_datasets["validation"] = split_datasets.pop("test")

In [38]:
epochs = 22
max_input_length = 256
max_target_length = 256
learning_rate = 1e-04
source_tag = "x"
target_tag = "y"
batch_size = 8
model_checkpoint = 'facebook/bart-base'
output_dir = "models/m3bart"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors='pt')

In [8]:
def preprocess_function(examples):
    inputs = [ex[source_tag] for ex in examples["mitigation"]]
    targets = [ex[target_tag] for ex in examples["mitigation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_datasets = split_datasets.map(preprocess_function, batched = True, remove_columns=split_datasets["train"].column_names,)

Map:   0%|          | 0/1658 [00:00<?, ? examples/s]

/Users/louys/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3862: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/293 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
device = torch.device("mps")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [13]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [14]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=learning_rate)

In [15]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [16]:
from transformers import get_scheduler

num_train_epochs = epochs
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [17]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [18]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=max_target_length,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")
    
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/4160 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/37 [00:00<?, ?it/s]

/Users/louys/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:822: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_1aidzjezue/croot/pytorch_1687856425340/work/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/louys/anaconda3/lib/python3.11/site-packages/transformers/generation/beam_search.py:372: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_1aidzjezue/croot/pytorch_1687856425340/work/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  sent_lengths_max = sent_lengths.max().item() + 1


epoch 0, BLEU score: 19.32


...

epoch 17, BLEU score: 17.65


  0%|          | 0/37 [00:00<?, ?it/s]

epoch 18, BLEU score: 17.31


  0%|          | 0/37 [00:00<?, ?it/s]

epoch 19, BLEU score: 17.32


In [19]:
# accelerator.wait_for_everyone()


('models/m3bart/tokenizer_config.json',
 'models/m3bart/special_tokens_map.json',
 'models/m3bart/vocab.json',
 'models/m3bart/merges.txt',
 'models/m3bart/added_tokens.json',
 'models/m3bart/tokenizer.json')

In [29]:
model_to_generate = AutoModelForSeq2SeqLM.from_pretrained("models/m3bart")
model_to_generate = model_to_generate.to('mps')

In [37]:
test_examples = "She would have made a poor mother. As most career women are. "

inputs = tokenizer(test_examples, padding="max_length", truncation=True, return_tensors="pt")
input_ids = inputs.input_ids.to(model_to_generate.device)
attention_mask = inputs.attention_mask.to(model_to_generate.device)

outputs = model_to_generate.generate(input_ids=input_ids, attention_mask=attention_mask)
output_str = tokenizer.batch_decode(outputs,skip_special_tokens=True)
print(output_str)

/Users/louys/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['She may not have been well-suited for motherhood, as many career-']


/Users/louys/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'translation_text': "Using derogatory language in interactions isn't appropriate."}]

In [11]:
# args = Seq2SeqTrainingArguments(
#     model_name,
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True,
#     # fp16=True,
#     # push_to_hub=True,
# )

In [21]:
# args = Seq2SeqTrainingArguments(
#     output_dir="models",
#     num_train_epochs=epochs,
#     do_train=True,
#     do_eval=True,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     learning_rate=learning_rate,
#     warmup_steps=200,
#     weight_decay=0.001,
#     predict_with_generate=True,
#     logging_dir="logs",
#     logging_steps=10,
#     evaluation_strategy="steps",
#     save_total_limit=3,
#     generation_max_length=max_target_length,
#     generation_num_beams=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="rouge-1",
#     # disable_tqdm=True,
#     report_to="all"
# )

In [23]:
# import numpy as np
# 
# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [[label.strip()] for label in labels]
# 
#     return preds, labels
# 
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
# 
# #    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
# 
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
# 
#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
# 
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {"bleu": result["score"]}
# 
#     # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     # result["gen_len"] = np.mean(prediction_lens)
#     # result = {k: round(v, 4) for k, v in result.items()}
#     return result
# 


In [24]:
# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets['test'],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [ ]:
# trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
# trainer.save_model("models/bart-base-text-style-transfer")

In [ ]:
# model_to_generate = AutoModelForSeq2SeqLM.from_pretrained("models/bart-base-text-style-transfer")
# model_to_generate = model_to_generate.to('mps')

In [ ]:
# test_examples = "Because she's a skank"
# 
# inputs = tokenizer(test_examples, padding="max_length", truncation=True, return_tensors="pt")
# input_ids = inputs.input_ids.to(model_to_generate.device)
# attention_mask = inputs.attention_mask.to(model_to_generate.device)
# 
# outputs = model_to_generate.generate(input_ids=input_ids, attention_mask=attention_mask)
# output_str = tokenizer.batch_decode(outputs,skip_special_tokens=True)
# print(output_str)